В этом ноубуке используются обeченные тестовые модели, с их помощью происходит сэмплирование и далее закодированные категориальные переменные переводятся в изначальные. После чего замеряется качество.

Способ нормализации: quantile transformation для числовых признаков и ничего (Identity) для категориальных OHE

Добавление шума: ко всем признакам

In [1]:
%cd ..

/workspace/TabDDPM_copy/TabDDPM


/venv/main/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [1]:
import json
import pandas as pd
import torch
import matplotlib.pyplot as plt

from utils import *
from models.tabddpm_ON_QnI.tabddpm_ON_QnI import *

from tqdm.notebook import tqdm

from eval.base_metrics import calculate_base_metrics
from eval.similarity import calculate_similarity
from eval.mle import calculate_mle
from eval.alpha_beta import calculate_alpha_beta
from eval.detection import calculate_detection
from eval.dcr import calculate_DCR

### Подготовка

In [2]:
dataname = 'adult_ON_QnI'
model_short = 'tabddpm_ON_QnI'

In [3]:
if torch.cuda.is_available():
    device = f'cuda'
else:
    device = 'cpu'
CONFIG.add_arg('device', device)
print(f"Device: {device}")

CONFIG.add_arg('dataname', dataname)
CONFIG.add_arg('real_path',
                   f'./synthetic/{dataname}/initial_real.csv')
CONFIG.add_arg('test_path',
                   f'./synthetic/{dataname}/initial_test.csv')

CONFIG.add_arg('info_path',
                   f'./data/{dataname}/initial_info.json')

Device: cuda


In [4]:
model_save_path_hist = []
for sigma in [0, 0.001, 0.01, 0.1, 0.25, 0.5]:
    model_save_path=f"./models/{model_short}/ckpt/{model_short}_const_{str(sigma).replace('.', '_')}"
    model_save_path_hist.append(model_save_path)

for i, sigmas in enumerate(range(10)):
    model_save_path=f"./models/{model_short}/ckpt/{model_short}_mult_{i}"
    model_save_path_hist.append(model_save_path)

In [5]:
from pickle import dump, load
with open(f'data/{dataname}/normalizers.json', 'r') as f:
        normalizers = json.load(f)
normalizers['num_normalizer'] = load(open(f'./data/{dataname}/num_normalizer_{dataname}.pkl', 'rb'))
normalizers['cat_normalizer'] = load(open(f'./data/{dataname}/cat_normalizer_{dataname}.pkl', 'rb'))
print(f"normalizers received from `data/{dataname}` folder")

normalizers received from `data/adult_ON_QnI` folder


In [6]:
normalizers

{'len_num_prev': 6,
 'len_cat_prev': 102,
 'len_target_prev': 1,
 'num_normalizer': QuantileTransformer(output_distribution='normal', random_state=0,
                     subsample=1000000000),
 'cat_normalizer': FunctionTransformer(func=<function Identity_transform at 0x7f2f1b25ff40>,
                     inverse_func=<function Identity_transform at 0x7f2f1b25ff40>)}

### Sample + Eval

In [7]:
CONFIG.get_all_args()

{'dataname': 'adult_ON_QnI',
 'method': None,
 'device': 'cuda',
 'mode': 'train',
 'train': 1,
 'sample_save_path': None,
 'sigma_scheduller_name': 'constant',
 'sigma_value': 0.001,
 'num_noise': 103,
 'real_path': './synthetic/adult_ON_QnI/initial_real.csv',
 'test_path': './synthetic/adult_ON_QnI/initial_test.csv',
 'info_path': './data/adult_ON_QnI/initial_info.json',
 'save_path': None}

In [ ]:
overall_metrics = {}

for model_save_path in model_save_path_hist:
    model_name = model_save_path.split('/')[-1]
    sample_save_path = f'./synthetic/{dataname}/{model_name}.csv'

    CONFIG.add_arg('method', model_name)
    CONFIG.add_arg('sample_save_path',
                       f"synthetic/{CONFIG.get_arg('dataname')}/{CONFIG.get_arg('method')}.csv")

    overall_metrics[model_name] = {}

    print(model_save_path, model_name, sample_save_path)

    tabddpm_noise_ohe = TabDDPM_OHE_Noise_QnI(CONFIG, model_save_path=model_save_path, sigmas=None)
    tabddpm_noise_ohe.sample(sample_save_path=sample_save_path)
    postsample_OHE(dataname, f'./synthetic/{dataname}/initial_{model_name}.csv', normalizers=normalizers)

    # подсчет метрик
    try:
        overall_metrics[model_name]['base_metrics'] = calculate_base_metrics(make_binary=True, value=' >50K') # ошибка
    except Exception as e:
        overall_metrics[model_name]['base_metrics'] = np.nan
    overall_metrics[model_name]['similarity'] = calculate_similarity()
    overall_metrics[model_name]['mle'] = calculate_mle()
    overall_metrics[model_name]['detection'] = calculate_detection()
    overall_metrics[model_name]['DCR'] = calculate_DCR()
    overall_metrics[model_name]['quality'] = calculate_alpha_beta()

./models/tabddpm_ON_QnI/ckpt/tabddpm_ON_QnI_const_0 tabddpm_ON_QnI_const_0 ./synthetic/adult_ON_QnI/tabddpm_ON_QnI_const_0.csv
No NaNs in numerical features, skipping
mlp
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Shape torch.Size([32561, 109])
Sampling time: 195.84394574165344
Sample path changed to ./synthetic/adult_ON_QnI/initial_tabddpm_ON_QnI_const_0.csv


/venv/main/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


CALCULATING COLUMN-WISE DENSITY ESTIMATION & PAIR-WISE COLUMN CORRELATION...
Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 15/15 [00:00<00:00, 68.39it/s]|
Column Shapes Score: 95.69%

(2/2) Evaluating Column Pair Trends: |██████████| 105/105 [00:01<00:00, 67.56it/s]|
Column Pair Trends Score: 91.35%

Overall Score (Average): 93.52%

Error rate (%) of column-wise density estimation TABDDPM_ON_QNI_CONST_0: 4.313 ± 4.391
Error rate (%) of pair-wise column correlation score TABDDPM_ON_QNI_CONST_0: 8.647 ± 6.158


DONE! 

DRAW COLUMN VALUES DISTRIBUTIONS...


DONE!


...

TABDDPM_ON_QNI_MULT_1: ADULT_ON_QNI Dataset
Original Logistic: 0.657
Synthetic Logistic: 0.293
Original Tree: 0.660
Synthetic Tree: 0.173 

TABDDPM_ON_QNI_MULT_1 Accuracy Loss: ADULT_ON_QNI
Logistic: 55.340%
Tree: 73.796% 

CALCULATING COLUMN-WISE DENSITY ESTIMATION & PAIR-WISE COLUMN CORRELATION...
Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 15/15 [00:00<00:00, 92.39it/s]| 
Column Shapes Score: 52.09%

(2/2) Evaluating Column Pair Trends: |██████████| 105/105 [00:01<00:00, 55.21it/s]|
Column Pair Trends Score: 35.18%

Overall Score (Average): 43.63%

Error rate (%) of column-wise density estimation TABDDPM_ON_QNI_MULT_1: 47.914 ± 16.921
Error rate (%) of pair-wise column correlation score TABDDPM_ON_QNI_MULT_1: 64.819 ± 30.384


DONE! 

DRAW COLUMN VALUES DISTRIBUTIONS...


DONE!


100%|██████████| 36/36 [00:11<00:00,  3.06it/s]


Saving scores to  eval/mle/adult_ON_QnI/tabddpm_ON_QnI_mult_1.json
ROC - AUC обучения на синтетических данных TABDDPM_ON_QNI_MULT_1: 0.511 ± 0.009
adult_ON_QnI, tabddpm_ON_QnI_mult_1: 0.0010581179784399097
DCR Score, a value closer to 0.5 is better
adult_ON_QnI-tabddpm_ON_QnI_mult_1, DCR Score = 0.6608826510242315
=========== All Features ===========
Data shape:  (32561, 110)
alpha precision: 0.023132, beta recall: 0.000799
./models/tabddpm_ON_QnI/ckpt/tabddpm_ON_QnI_mult_2 tabddpm_ON_QnI_mult_2 ./synthetic/adult_ON_QnI/tabddpm_ON_QnI_mult_2.csv
No NaNs in numerical features, skipping
mlp
Sample timestep  463

In [ ]:
final_metrics_table = []
for m in overall_metrics.keys():
    # Сбор таблички результатов
    tmp = pd.DataFrame([{'Model':model_short, 'Type':m, 'Data':dataname}])
    tmp.columns = pd.MultiIndex.from_tuples([('', i) for i in tmp.columns])
    result = [tmp]
    
    for metric_group in overall_metrics[m].keys():
        tmp = pd.DataFrame([overall_metrics[m][metric_group]])
        tmp.columns = pd.MultiIndex.from_tuples([(metric_group, i) for i in tmp.columns])
        result.append(tmp)
    result = pd.concat(result, axis = 1)
    final_metrics_table.append(result)
    
final_metrics_table = pd.concat(final_metrics_table)

In [ ]:
pd.set_option('display.max_columns', None)
final_metrics_table.round(3).drop(columns=('base_metrics', 0)).sort_values([('similarity', 'Overall Score (Average), %')],
                                                                          ascending=False)

In [ ]:
import os
if not os.path.exists(f'./eval/total/{dataname}'):
    os.makedirs(f'./eval/total/{dataname}')
final_metrics_table.to_csv(f'./eval/total/{dataname}/{model_short}_final_metrcs_table.csv', index=False)